In [10]:
# Imports
from qiime2 import Metadata
import pandas as pd
from qiime2 import Artifact, sdk
from qiime2.plugins.dada2.methods import denoise_pyro # The samples were obtained through pyrosequencing
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.feature_classifier.pipelines import classify_consensus_blast
import qiime2.plugins.feature_table.methods as ftm
import qiime2.plugins.feature_classifier.actions as feature_classifier_actions
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
import qiime2.plugins.feature_table.actions as fta
import matplotlib.pyplot as plt
import matplotlib 
import seaborn as sns
from Bio import SeqIO

matplotlib.use('module://ipykernel.pylab.backend_inline')
pm = sdk.PluginManager()
def see(artifact):
    from_format = artifact.format
    if issubclass(from_format, sdk.plugin_manager.SingleFileDirectoryFormatBase):
        from_format = artifact.format.file.format
    return set(pm.transformers[from_format].keys())
import os
import pandas
import qiime2
import tempfile

def v2frame(viz_fp: str) -> list:
    '''viz_fp is a path to the qiime2 visualization object'''
    viz = qiime2.Visualization.load(viz_fp)
    with tempfile.TemporaryDirectory() as tmpdir:
        viz.export_data(tmpdir)
        fp = os.path.join(tmpdir, 'quality-plot.html')
        ov = os.path.join(tmpdir, 'overview.html')
        dfs = pandas.read_html(fp, index_col=0)
        df2s = pandas.read_html(ov, index_col=0)
    return dfs + df2s

# Importing data from fastq

# Initial exploration

# Data cleaning

# Sequence clustering

# Taxonomic analyses
For sample taxonomic classification, I will be trying out all three of the methods available in qiime2: 

#### Database preparation
Two databases were obtained:

- All annotated 16S rRNA from the NCBI ftp [website](https://ftp-ncbi-nlm-nih-gov.ejournal.mahidol.ac.th/blast/db/v5/)
- Annotated 16S rRNA used by the tool [MicFunPred](https://github.com/microDM/MicFunPred)
- A 16S rRNA database from [EzBioCloud](https://www.ezbiocloud.net/dashboard)

The MicFunPred and NCBI databases were obtained in the standard BLAST format, and need to be converted into compatible data types for import into the qiime2 workflow. Specficially, I needed to convert them into FASTA format with an associated taxonomy mapping file (in HeaderlessTSVTaxonomyFormat)
- Steps
    - 1 Extract all entries from the database in FASTA format
    - 2 Extract the header and convert it into the HeaderLessTSVTaxonomyFormat, which is a tab-delimited file of FASTA identifiers followed by their taxonomic assignments
    - 3 Remove the taxonomic assignments from the original FASTA file
    - 4 Concatenate the respective files types together, then import them files as qiime2 Artifacts
    - 5 Repeat for the other database (if both databases used the identifier conventions I could have combined them and processed them together but unfortunately this was not the case )

```bash
# 1
blastdbcmd -db NCBI_16S/16S_ribosomal_RNA -entry all > NCBI_16S.fasta
blastdbcmd -db micfun/micfun16S -entry all > micfun.fasta

# 2 
grep '>' NCBI_16S.fasta | tr -d '>' | sed 's/ /\t/' | sed 's/ /_/g' > NCBI_16SID.txt
grep '>' micfun.fasta | tr -d '>' | sed 's/_/\t/' | sort | uniq > micfunID.txt # Unfortunately several of the headers repeat 

# 3 
cat micfun.fasta | sed 's/_.*//' > micfunID.fasta
cat NCBI_16S.fasta | sed 's/ .*//' > ncbi16sID.fasta

# 4 
cat micfunID.txt NCBI_16SID.txt EzBioCloud/ezbiocloud_id_taxonomy.txt > all_mappings.txt
cat EzBioCloud/ezbiocloud_qiime_full.fasta ncbi16sID.fasta micfun.fasta > all.fasta
qiime tools import     --type FeatureData[Taxonomy]     --input-format HeaderlessTSVTaxonomyFormat     --input-path all.fasta     --output-path all_fasta.qza
qiime tools import     --type FeatureData[Taxonomy]     --input-format HeaderlessTSVTaxonomyFormat     --input-path all_uniqIDs.txt     --output-path Ids.qza
```

- Altogether, the database contains 130,122 sequences (though there might be some repetition that was overlooked)

In [ ]:
# Script for removing redundant ids
from Bio import SeqIO
import csv

exists: set = set()
mapped = open('all_uniq.fasta', 'w+')
for seq in SeqIO.parse('all.fasta', 'fasta'):
    if seq.id in exists:
        continue
    exists.add(seq.id)
    mapped.write(f'>{seq.id}\n')
    mapped.write(f'{seq.seq}\n')
mapped.close

uniq = open('all_uniqIDs.txt', 'w+')
exists2: set = set()
with open('all_mappings.txt', 'r') as i:
    for id in csv.reader(i, delimiter='\t'):
        if id[0] in exists2:
            continue
        exists2.add(id[0])
        uniq.write(f'{id[0]}\t{id[1]}\n')
uniq.close


#### Merging with other data
The qiime2 website provides links for data resources, including 16s rRNA reference sequences. These are from the SILVA and greengenes databases

In [12]:
# Import database artifacts
IDs = Artifact.load('data/artifacts/Ids.qza') 
RefSeqs = Artifact.load('data/artifacts/all_fasta.qza')
silvaSeqs = Artifact.load('data/downloads/silva-138-99-seqs.qza')
silvaIDs = Artifact.load('data/downloads/silva-138-99-tax.qza')
greenSeqs = Artifact.load('data/downloads/2022.10.backbone.full-length.fna.qza')
greenIDs = Artifact.load('data/downloads/2022.10.backbone.tax.qza')
AllIDs = ftm.merge_taxa([IDs, silvaIDs, greenIDs])
AllSeqs = ftm.merge_seqs([RefSeqs, silvaSeqs, greenSeqs])
AllIDs.merged_data.save('data/artifacts/AllIDs.qza')
AllSeqs.merged_data.save('data/artifacts/AllSeqs.qza')

/home/sc31/Bio_SDD/miniconda3/envs/qiime2-2023.2/lib/python3.8/site-packages/q2_types/feature_data/_transformer.py:258: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for id_, seq in data.iteritems():


### Taxonomic classification


# Diversity analyses

# References

# Load artifacts

In [ ]:
AllIDs = Artifact.load('data/artifacts/AllIDs.qza')
AllSeqs = Artifact.load('data/artifacts/AllSeqs.qza')